# Lab 7 - Recommender Systems
CS 437  
Fall 2025  
Dr. Henderson  
_v1_

---

In this lab you will implement a movie recommender system using K-nearest neighbors. We will use a simple list to represent a user's vector of ratings.

The first step is to define our distance measure. We'll use a simple Euclidean distance squared measure:

$$ distance(X, Y) = || (X - Y) ||^2 = (X - Y) \cdot (X - Y) $$

1. Create a function named `distance()` which takes two vectors and returns the distance between them.

In [1]:
def distance(X, Y):
    return sum( [ (X[i] - Y[i]) ** 2 for i in range(len(X)) ] )

Run the unit tests below to test your function

In [2]:
_ut1_1 = distance([4,4,4], [4,4,4])
_ut1_2 = distance([4,4,4], [5,5,5])
_ut1_3 = distance([4,4,4], [1,1,1])
assert _ut1_1 == 0, f"Incorrect distance: {_ut1_1}"
assert _ut1_2 == 3, f"Incorrect distance: {_ut1_2}"
assert _ut1_3 == 27, f"Incorrect distance: {_ut1_3}"

Once we have a distance function the K-nearest neighbors algorithm is pretty simple. To find the k-nearest neighbors for user $u$ calculate the distance between $u$ and every other user and return the $k$ highest scoring users.

2. Create a function called `neighbors()` which takes a user id (integer index), a parameter $k$, and a list of user ratings vectors. The function should return a list of user ids which are the $k$ nearest neighbors, i.e. the smallest distance.

In [3]:
def neighbors(u, k, ratings):
    results = []
    ur = ratings[u]
    for _id in range(len(ratings)):
        if _id != u: results.append( (distance(ur, ratings[_id]), _id) )
    return [ x[1] for x in sorted(results)[:k] ]

Once we have the neighbors for a user we need some way to combine their ratings into a predicted rating. We could just average the ratings of the neighbors but this ignores the distances (similarities) of each neighbor. Instead we can weight each neighbor's (relevant) rating by their similarity with the user, sum these over all neighbors and normalize by the total similarity:



$$ \hat{r}_u(i) = \bar{r}_u + \frac{1}{ \sum_{u' \in N(u)} sim(u, u') } \sum_{u' \in N(u)} sim(u, u')( r_{u'}(i) - \bar{r}_{u'}) $$

3. Implement the prediction formula above by creating a function named `predict()` that takes a user id $uid$, a movie id $mid$, a list of neighbor ids, and a dictionary of user id -> user ratings vectors. The function should return a predicted rating for the movie with id $mid$.

In [4]:
def predict(uid, mid, neighbors, ratings):
    rs = [ x for x in ratings[uid] if x != 0 ]
    rbar = sum(rs) / len(rs)
    norm = 0
    wr = 0
    for nid in neighbors:
        nrs = [ x for x in ratings[uid] if x != 0 ]
        nbar = sum(nrs) / len(nrs)
        s = 1 / distance(ratings[uid], ratings[nid])
        norm += s
        wr += s * (ratings[nid][mid] - rbar)
    return rbar + (1 / norm) * wr

The `movie-ratings-vectors.csv` file contains the ratings from 610 users. Each user is represented as a line of comma-separated ratings. Movies not rated by the user will contain a 0 value.

4. Create a list called `rankings`. Read the file `movie-ratings-vectors.csv` and add each line from the file to `rankings` as a vector (essentially another list). Show the first 10 entries for the first vector.

In [5]:
rankings = []
with open("movie-ratings-vectors.csv") as file:
    for line in file.readlines():
        rankings.append(list(map(float, line.split(','))))
rankings[0][:10]

[4.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0]

We can evaluate our system by comparing predicted ratings to ratings that already exist.

5. Create a function named `predicted_mse()` which takes a user id, a list of neighbors, and a list of ratings vectors. The function should calculate the MSE by iterating through all non-zero ratings of the given user, calling the `predict()` function and summing the squares of the differences. The function should return the average of the sum.

In [6]:
def predicted_mse(uid, neighbors, ratings):
    err = 0
    errs = 0
    for mid, rating in enumerate(ratings[uid]):
        if rating != 0:
            pv = predict(uid, mid, neighbors, ratings)
            err += (pv - rating) ** 2
            errs += 1
    return err / errs

6. For the first user in the ratings list, find the five nearest neighbors and give the MSE of the resulting predictions.

In [7]:
neighbors(0, 5, rankings)

[492, 38, 493, 207, 179]

In [8]:
predicted_mse(0, neighbors(0, 5, rankings), rankings)

16.728721198120382

7. Repeat for 10 neighbors

In [9]:
predicted_mse(0, neighbors(0, 10, rankings), rankings)

16.906756814781986

Now it's time to make a prediction!

For the first user, iterate through the ratings and for each 0 call the `predict()` function using your $k = 5$ neighbor array. Stop if you find a predicted rating above 4, otherwise return the movie with the highest predicted rating.

In [10]:
u0_neighbors = neighbors(0, 5, rankings)

uid = 0
best = (0, 0)
for mid, rating in enumerate(rankings[uid]):
    if rating == 0:
        pv = predict(uid, mid, u0_neighbors, rankings)
        if pv >= 4:
            best = pv, mid
            break
        if pv > best[0]:
            best = (pv, mid)

best

(3.405344044937813, 507)

The function `get_movie(index)` from `lab7lib` will return a dictionary of information about the movie at `index`. Call it with the index from your prediction above to see the predicted move.

In [11]:
import lab7lib
from importlib import reload
reload(lab7lib)
lab7lib.get_movie(507)

{'movieId': 589,
 'title': 'Terminator 2: Judgment Day',
 'genres': 'Action|Sci-Fi',
 'year': 1991.0,
 'index': 507}

---

### Submission Instructions

Be sure to ***SAVE YOUR WORK***!  

Next, select Kernel -> Restart Kernel and Run All Cells...

Make sure there are no errors.

Use _File > Save and Export Notebook As > HTML_ then submit your HTML file to Canvas.